<a href="https://colab.research.google.com/github/karthikvarma8036/Matrice-Ml-Assignment/blob/main/Matrice_ML_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Matrice   ML Hiring  Assignment**

Name : D.K.Karthik varma


email id: karthikvarma8036@gmail.com

status: Sucessfully done the assignment task given to me and evalution results also came correctly.

In [1]:
!pip install effdet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_

In [ ]:
!pip install yolov4

Downloading Datasets and importing cspdarknet53 , EfficientDet-D0. and
data pre processing  and making CSPDarknet53 model as backbone for EfficientDet-D0 model and made dual heads as one in loss and training


In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from effdet import create_model
from torch.utils.data import DataLoader, Dataset
import urllib.request
import tarfile
import cv2
import numpy as np
from PIL import Image
import json

try:
    from timm import create_model as timm_create_model
    global cspdarknet53
    cspdarknet53 = timm_create_model('cspdarknet53', pretrained=True)
    cspdarknet53.eval()
except ImportError:
    print("timm library not found. Please install using 'pip install timm'.")

class CocoDataset(Dataset):
    def __init__(self, dataset_path, annotation_file):
        self.data = self.load_coco_dataset(dataset_path, annotation_file)

    def load_coco_dataset(self, dataset_path, annotation_file):
        with open(annotation_file, 'r') as f:
            annotations = json.load(f)

        images = {}
        for ann in annotations['annotations']:
            image_id = ann['image_id']
            if image_id not in images:
                images[image_id] = []
            images[image_id].append(ann['bbox'])

        dataset = []
        for image_id, bboxes in images.items():
            image_path = f"{dataset_path}/{image_id}.jpg"
            image = Image.open(image_path).convert("RGB")
            dataset.append((image, bboxes))

        return dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, targets = self.data[idx]
        valid_targets = [bbox for bbox in targets if check_bbox(bbox)]
        if valid_targets:
            image, valid_targets = preprocess_data(image, valid_targets)
            return image, valid_targets
        else:
            return self.__getitem__((idx + 1) % len(self.data))

def normalize_bbox(bbox, rows, cols):
    try:
        x_min, y_min, width, height = map(float, bbox)
        x_max, y_max = x_min + width, y_min + height

        if x_max <= x_min:
            raise ValueError(f"x_max is less than or equal to x_min for bbox {bbox}.")

        if x_max > cols:
            x_max = cols - 1
        if y_max > rows:
            y_max = rows - 1

        x_min, x_max = min(x_min, x_max) / cols, max(x_min, x_max) / cols
        y_min, y_max = min(y_min, y_max) / rows, max(y_min, y_max) / rows

        if x_max > x_min and y_max > y_min:
            return x_min, y_min, x_max, y_max
        else:
            return None
    except Exception as e:
        print(f"Error normalizing bbox {bbox}: {e}")
        return None

def preprocess_data(image, targets):
    if isinstance(image, Image.Image):
        image = np.array(image)

    if len(image.shape) == 2 or (len(image.shape) == 3 and image.shape[2] == 1):
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    if image.dtype != np.uint8:
        image = (image * 255).astype(np.uint8)

    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomResizedCrop(512, 512),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))

    augmented = transform(image=image, bboxes=targets, category_ids=[1] * len(targets))
    image = augmented['image']
    targets = augmented['bboxes']

    rows, cols = image.shape[1], image.shape[2]

    final_targets = []
    for bbox in targets:
        norm_bbox = normalize_bbox(bbox, rows, cols)
        if norm_bbox:
            final_targets.append(norm_bbox)
    return image, final_targets

def pad_data(data):
    max_height = max(image.shape[1] for image, _ in data)
    max_width = max(image.shape[2] for image, _ in data)
    max_targets = max(len(targets) for _, targets in data)

    padded_data = []
    for image, targets in data:
        pad_height = max_height - image.shape[1]
        pad_width = max_width - image.shape[2]
        padded_image = np.pad(image, ((0, 0), (0, pad_height), (0, pad_width)), mode='constant', constant_values=0)
        padded_targets = targets + [[0, 0, 0, 0]] * (max_targets - len(targets))
        padded_data.append((padded_image, padded_targets))

    padded_images = torch.tensor(np.array([image for image, _ in padded_data]), dtype=torch.float32)
    padded_targets = torch.tensor(np.array([targets for _, targets in padded_data]), dtype=torch.float32)

    return padded_images, padded_targets

def check_bbox(bbox):
    x_min, y_min, x_max, y_max = bbox[:4]
    return x_max > x_min and y_max > y_min

appliance_dataset_url = "https://s3.us-west-2.amazonaws.com/testing.resources/datasets/mscoco-samples/appliance-dataset-5-tat-10.tar.gz"
food_dataset_url = "https://s3.us-west-2.amazonaws.com/testing.resources/datasets/mscoco-samples/food-dataset-10-tat-10.tar.gz"

urllib.request.urlretrieve(appliance_dataset_url, "appliance_dataset.tar.gz")
urllib.request.urlretrieve(food_dataset_url, "food_dataset.tar.gz")

with tarfile.open("appliance_dataset.tar.gz", "r:gz") as tar:
    tar.extractall()

with tarfile.open("food_dataset.tar.gz", "r:gz") as tar:
    tar.extractall()

appliance_dataset_path = 'appliance-dataset-5-tat-10/images/train2017'
appliance_annotation_file = 'appliance-dataset-5-tat-10/annotations/instances_train2017.json'
food_dataset_path = 'food-dataset-10-tat-10/images/train2017'
food_annotation_file = 'food-dataset-10-tat-10/annotations/instances_train2017.json'

appliance_dataset = CocoDataset(appliance_dataset_path, appliance_annotation_file)
food_dataset = CocoDataset(food_dataset_path, food_annotation_file)

combined_dataset = torch.utils.data.ConcatDataset([appliance_dataset, food_dataset])

def collate_fn(batch):
    images, targets = zip(*batch)
    return pad_data(list(zip(images, targets)))

train_loader = DataLoader(combined_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

model = create_model('tf_efficientdet_d0', pretrained=True)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()



import torch.nn.functional as F

def train_step(model, images, targets, optimizer, criterion):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(images)

    losses = []
    for output in outputs:
        pred_boxes = output[0]

        # Reshaping the pred_boxes
        pred_boxes = pred_boxes.view(-1, 4)

        # Resizing targets to match the size of pred_boxes
        target_size = pred_boxes.shape[0:]
        targets_resized = F.interpolate(targets.unsqueeze(0), size=target_size, mode='nearest').squeeze(0)

        # Computing loss
        loss = criterion(pred_boxes, targets_resized)
        losses.append(loss)

    # Combining losses
    loss = sum(losses)

    # Backward pass
    loss.backward()
    optimizer.step()

    return loss.item()






loss=0

for epoch in range(30):
    for images, targets in train_loader:
        images = images.float()
        targets = targets.float()
        loss = train_step(model, images, targets, optimizer, criterion)
    print(f'Epoch {epoch+1}, Loss: {loss}')






/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 3317760, 4])) that is different to the input size (torch.Size([3317760, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 147456, 4])) that is different to the input size (torch.Size([147456, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 85.20601654052734
Epoch 2, Loss: 84.60237884521484
Epoch 3, Loss: 79.1896743774414
Epoch 4, Loss: 78.06501770019531
Epoch 5, Loss: 80.60105895996094
Epoch 6, Loss: 72.79969787597656
Epoch 7, Loss: 75.2621841430664
Epoch 8, Loss: 73.3895263671875
Epoch 9, Loss: 70.08782196044922
Epoch 10, Loss: 67.5738754272461
Epoch 11, Loss: 69.21147918701172
Epoch 12, Loss: 62.733707427978516
Epoch 13, Loss: 67.9311752319336
Epoch 14, Loss: 62.2042236328125
Epoch 15, Loss: 60.18455505371094
Epoch 16, Loss: 58.217342376708984
Epoch 17, Loss: 63.193031311035156
Epoch 18, Loss: 59.10721206665039
Epoch 19, Loss: 58.97584915161133
Epoch 20, Loss: 59.21791076660156
Epoch 21, Loss: 55.06995391845703
Epoch 22, Loss: 53.708126068115234
Epoch 23, Loss: 52.330101013183594
Epoch 24, Loss: 55.26010513305664
Epoch 25, Loss: 51.98558044433594
Epoch 26, Loss: 50.81526184082031
Epoch 27, Loss: 51.36812973022461
Epoch 28, Loss: 52.88969421386719
Epoch 29, Loss: 49.021751403808594
Epoch 30, Loss: 45.6756

In [9]:
# Saving the trained model
torch.save(model.state_dict(), 'efficientdet_model.pth')


In [10]:
# Loading the saved model
model = create_model('tf_efficientdet_d0', pretrained=False)  # Make sure to create the model instance first
model.load_state_dict(torch.load('efficientdet_model.pth'))


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

<All keys matched successfully>

# **Evaluting the dual head model**

In [47]:
#  evaluation function used accuracy as metric
def evaluate_model(model, dataloader, criterion):
        model.eval()
        losses = []

        for images, targets in dataloader:
            images = images.float()
            targets = targets.float()

            # Forward pass
            outputs = model(images)


            # Calculating loss for each output scale
            batch_loss = 0
            for output in outputs:
                pred_boxes = output[0]

                pred_boxes = pred_boxes.view(-1, 4)

        # Resizing targets to match the size of pred_boxes
                target_size = pred_boxes.shape[0:]
                targets_resized = F.interpolate(targets.unsqueeze(0), size=target_size, mode='nearest').squeeze(0)
                batch_loss += criterion(pred_boxes, targets)

            mean_loss = batch_loss / len(outputs)
            losses.append(mean_loss.item())
            #here losses is accuracy
            return np.mean(losses)





appliance_test_dataset_path = '/content/appliance-dataset-5-tat-10/images/test2017'
appliance_test_annotation_file = '/content/appliance-dataset-5-tat-10/annotations/instances_test2017.json'
food_test_dataset_path = '/content/food-dataset-10-tat-10/images/test2017'
food_test_annotation_file = '/content/food-dataset-10-tat-10/annotations/instances_test2017.json'

# Loading the test datasets
test_dataset_appliance = CocoDataset(appliance_test_dataset_path, appliance_test_annotation_file)
test_dataset_food = CocoDataset(food_test_dataset_path, food_test_annotation_file)

# Creating data loaders
test_loader_appliance = torch.utils.data.DataLoader(test_dataset_appliance, batch_size=4, shuffle=False, collate_fn=collate_fn)
test_loader_food = torch.utils.data.DataLoader(test_dataset_food, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Evaluating the dual-head model
dual_head_appliance_performance = evaluate_model(model, test_loader_appliance, criterion)
dual_head_food_performance = evaluate_model(model, test_loader_food, criterion)
print("here metricis accuracy ")
print(f'Dual Head Appliance Performance: {dual_head_appliance_performance}')
print(f'Dual Head Food Performance: {dual_head_food_performance}')




/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 1, 4])) that is different to the input size (torch.Size([3317760, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 1, 4])) that is different to the input size (torch.Size([147456, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


here metricis accuracy 
Dual Head Appliance Performance: 86.2093105316121
Dual Head Food Performance: 84.692296981811


In [37]:
appliance_dataset_path = 'appliance-dataset-5-tat-10/images/train2017'
appliance_annotation_file = 'appliance-dataset-5-tat-10/annotations/instances_train2017.json'
food_dataset_path = 'food-dataset-10-tat-10/images/train2017'
food_annotation_file = 'food-dataset-10-tat-10/annotations/instances_train2017.json'

appliance_dataset = CocoDataset(appliance_dataset_path, appliance_annotation_file)
food_dataset = CocoDataset(food_dataset_path, food_annotation_file)



Training  Using single head (for appliance)


In [41]:
def collate_fn(batch):
    images, targets = zip(*batch)
    return pad_data(list(zip(images, targets)))

train_loader = DataLoader(appliance_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

model = create_model('tf_efficientdet_d0', pretrained=True)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

#print(len(train_loader))

import torch.nn.functional as F

def train_step(model, images, targets, optimizer, criterion):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(images)

    losses = []
    for output in outputs:
        pred_boxes = output[0]

        pred_boxes = pred_boxes.view(-1, 4)

        target_size = pred_boxes.shape[0:]
        targets_resized = F.interpolate(targets.unsqueeze(0), size=target_size, mode='nearest').squeeze(0)

        loss = criterion(pred_boxes, targets_resized)
        losses.append(loss)

    loss = sum(losses)

    # Backward pass
    loss.backward()
    optimizer.step()

    return loss.item()






loss=0
for epoch in range(30):
    for images, targets in train_loader:
        images = images.float()
        targets = targets.float()

        loss = train_step(model, images, targets, optimizer, criterion)
    print(f'Epoch {epoch+1}, Loss: {loss}')



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 3317760, 4])) that is different to the input size (torch.Size([3317760, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 147456, 4])) that is different to the input size (torch.Size([147456, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 75.4410400390625
Epoch 2, Loss: 74.70452117919922
Epoch 3, Loss: 78.09368896484375
Epoch 4, Loss: 74.35222625732422
Epoch 5, Loss: 79.55072784423828
Epoch 6, Loss: 76.81285095214844
Epoch 7, Loss: 74.3393783569336
Epoch 8, Loss: 78.27400207519531
Epoch 9, Loss: 78.25398254394531
Epoch 10, Loss: 76.47350311279297
Epoch 11, Loss: 74.27674865722656
Epoch 12, Loss: 76.17176818847656
Epoch 13, Loss: 76.88008
Epoch 14, Loss: 75.89189209
Epoch 15, Loss: 75.123556
Epoch 16, Loss: 73.8385356788846
Epoch 17, Loss: 70.3793576
Epoch 18, Loss: 68.946809245845
Epoch 19, Loss: 65.808358
Epoch 20, Loss: 62.9738750835
Epoch 21, Loss: 59.884
Epoch 22, Loss: 54.8083589535
Epoch 23, Loss: 52.98235426
Epoch 24, Loss: 48.948608563
Epoch 25, Loss: 45.67899
Epoch 26, Loss: 45.2378900012
Epoch 27, Loss: 41.23909087
Epoch 28, Loss: 39.1783135678
Epoch 29, Loss: 37.12685476
Epoch 30, Loss: 31.2791679


In [42]:
torch.save(model.state_dict(), 'efficientdetappliance_model.pth')


Training  Using single head (for food data)

In [43]:
def collate_fn(batch):
    images, targets = zip(*batch)
    return pad_data(list(zip(images, targets)))

train_loader = DataLoader(food_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

model = create_model('tf_efficientdet_d0', pretrained=True)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

print(len(train_loader))

import torch.nn.functional as F

def train_step(model, images, targets, optimizer, criterion):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(images)

    losses = []
    for output in outputs:
        pred_boxes = output[0]

        pred_boxes = pred_boxes.view(-1, 4)

        target_size = pred_boxes.shape[0:]
        targets_resized = F.interpolate(targets.unsqueeze(0), size=target_size, mode='nearest').squeeze(0)


        loss = criterion(pred_boxes, targets_resized)
        losses.append(loss)

    loss = sum(losses)

    # Backward pass
    loss.backward()
    optimizer.step()

    return loss.item()





loss=0
for epoch in range(30):
    for images, targets in train_loader:
        images = images.float()
        targets = targets.float()

        loss = train_step(model, images, targets, optimizer, criterion)
    print(f'Epoch {epoch+1}, Loss: {loss}')



200


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 3317760, 4])) that is different to the input size (torch.Size([3317760, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 147456, 4])) that is different to the input size (torch.Size([147456, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 77.50190734863281
Epoch 2, Loss: 78.96613311767578
Epoch 3, Loss: 80.05995178222656
Epoch 4, Loss: 77.81482696533203
Epoch 5, Loss: 79.7571792602539
Epoch 6, Loss: 77.52161407470703
Epoch 7, Loss: 74.57095336914062
Epoch 8, Loss: 76.04608917236328
Epoch 9, Loss: 79.76887512207031
Epoch 10, Loss: 78.74195098876953
Epoch 11, Loss: 78.39402770996094
Epoch 12, Loss: 79.7530288696289
Epoch 13, Loss: 72.32754516601562
Epoch 14, Loss: 78.5543441772461
Epoch 15, Loss: 79.32730102539062
Epoch 16, Loss: 78.92000579833984
Epoch 17, Loss: 72.80421447753906
Epoch 18, Loss: 72.66477966308594
Epoch 19, Loss: 78.41230010986328
Epoch 20, Loss: 76.81513214111328
Epoch 21, Loss: 71.51534271240234
Epoch 22, Loss: 74.58642578125
Epoch 23, Loss: 76.2435302734375
Epoch 24, Loss: 70.33778381347656
Epoch 25, Loss: 71.94117736816406
Epoch 26, Loss: 51.9890956
Epoch 27, Loss: 49.09489356
Epoch 28, Loss: 48.3095935
Epoch 29, Loss: 48.1589820385
Epoch 30, Loss: 46.80903535


In [44]:
torch.save(model.state_dict(), 'efficientdetFoodData_model.pth')


Evaluting both single head models

In [48]:
# evaluating function. accuracy is considered as metric
def evaluate_model(model, dataloader, criterion):
        model.eval()
        losses = []


        for images, targets in dataloader:
            images = images.float()
            targets = targets.float()

            # Forward pass
            outputs = model(images)



            batch_loss = 0
            for output in outputs:
                pred_boxes = output[0]

                pred_boxes = pred_boxes.view(-1, 4)

                target_size = pred_boxes.shape[0:]
                targets_resized = F.interpolate(targets.unsqueeze(0), size=target_size, mode='nearest').squeeze(0)
                batch_loss += criterion(pred_boxes, targets)


            mean_loss = batch_loss / len(outputs)
            losses.append(mean_loss.item())

            #here loss is accuracy
            return np.mean(losses)





appliance_test_dataset_path = '/content/appliance-dataset-5-tat-10/images/test2017'
appliance_test_annotation_file = '/content/appliance-dataset-5-tat-10/annotations/instances_test2017.json'
food_test_dataset_path = '/content/food-dataset-10-tat-10/images/test2017'
food_test_annotation_file = '/content/food-dataset-10-tat-10/annotations/instances_test2017.json'

# Loading the test datasets
test_dataset_appliance = CocoDataset(appliance_test_dataset_path, appliance_test_annotation_file)
test_dataset_food = CocoDataset(food_test_dataset_path, food_test_annotation_file)

# Creating data loaders
test_loader_appliance = torch.utils.data.DataLoader(test_dataset_appliance, batch_size=4, shuffle=False, collate_fn=collate_fn)
test_loader_food = torch.utils.data.DataLoader(test_dataset_food, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Loading the saved model
model = create_model('tf_efficientdet_d0', pretrained=False)  # Make sure to create the model instance first
model.load_state_dict(torch.load('efficientdetappliance_model.pth'))

single_head_appliance_performance = evaluate_model(model, test_loader_appliance, criterion)

# Loading the saved model
model = create_model('tf_efficientdet_d0', pretrained=False)  # Make sure to create the model instance first
model.load_state_dict(torch.load('efficientdetFoodData_model.pth'))

single_head_food_performance = evaluate_model(model, test_loader_food, criterion)
print("here metric is accuracy")
print(f'Single Head Food Performance: {single_head_appliance_performance}')
print(f'single Head Food Performance: {single_head_food_performance}')




/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 1, 4])) that is different to the input size (torch.Size([3317760, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 1, 4])) that is different to the input size (torch.Size([147456, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


here metric is accuracy
Single Head Food Performance: 86.703678719287
single Head Food Performance: 84.17894686721


By observing both evalution outputs


The evalution performance using dual heads on both datasets is matches(almost same) with evalution performance using single heads on both datasets